<a href="https://colab.research.google.com/github/alibakhtiyari99/Learning-Git-start/blob/main/SteelPriceApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import ttk
from ttkthemes import ThemedTk
import requests
from bs4 import BeautifulSoup
import pygame

Libraries That We need

In [ ]:
# Initialize pygame mixer for sound effects
pygame.mixer.init()

# Function to play a click sound
def play_click_sound():
    pygame.mixer.Sound("click.wav").play()

# URLs of the target pages
urls = {
    "AhanOnline": "https://ahanonline.com/product-category/%D9%85%DB%8C%D9%84%DA%AF%D8%B1%D8%AF/%D9%82%DB%8C%D9%85%D8%AA-%D9%85%DB%8C%D9%84%DA%AF%D8%B1%D8%AF/",
    "Pivan": "https://pivan.co/product-category/rebar/",
    "AhanPakhsh": "https://www.ahanpakhsh.com/round-bar/ribbed-round-bar"
}

# Headers to simulate a real browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

# Function to convert Persian numbers to English numbers
def convert_persian_to_english(persian_number):
    persian_digits = "۰۱۲۳۴۵۶۷۸۹"
    english_digits = "0123456789"
    translation_table = str.maketrans(persian_digits, english_digits)
    return persian_number.translate(translation_table)

# Function to extract prices from AhanOnline
def get_prices_from_ahanonline(url):
    response = requests.get(url, headers=headers)
    prices = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        price_elements = soup.find_all("div", class_="text-[#35332E] font-Bold product-price text-[18px]")
        for price in price_elements[:9]:
            price_text = price.text.strip()
            if price_text in ["-", "تماس بگیرید"]:
                prices.append(price_text)
            else:
                prices.append(convert_persian_to_english(price_text))
    return prices

# Function to extract prices from Pivan
def get_prices_from_pivan(url):
    response = requests.get(url, headers=headers)
    prices = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        price_elements = soup.find_all("span", class_="toggle-price-when-check font-black text-black ex-tax")
        for price in price_elements[:9]:
            price_text = price.text.strip()
            if price_text in ["-", "تماس بگیرید"]:
                prices.append(price_text)
            else:
                prices.append(convert_persian_to_english(price_text))
    return prices

# Function to extract prices from AhanPakhsh
def get_prices_from_ahanpakhsh(url):
    response = requests.get(url, headers=headers)
    prices = []
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        price_elements = soup.find_all("span", class_="hikashop_product_price hikashop_product_price_0")
        for price in price_elements[:9]:
            price_text = price.text.strip().replace(" تومان", "")
            if price_text in ["-", "تماس بگیرید"]:
                prices.append(price_text)
            else:
                prices.append(convert_persian_to_english(price_text))
    return prices

In the first part of the code, we use the `pygame` library to define a function that plays the "Click.wav" sound when called.  

Next, we specify the websites from which we want to retrieve data. After that, we define functions to extract the price from these websites using the `requests` library.  

To achieve this, we need to visit the website, right-click on the price, select `Inspect`, and then use the extracted information in our Python code.

In [ ]:
# Function to show prices based on the selected website and size
def show_prices():
    play_click_sound()  # Play click sound

    selected_website = website_combobox.get()
    selected_size = size_combobox.get()

    if selected_website == "AhanOnline":
        prices = get_prices_from_ahanonline(urls["AhanOnline"])
    elif selected_website == "Pivan":
        prices = get_prices_from_pivan(urls["Pivan"])
    elif selected_website == "AhanPakhsh":
        prices = get_prices_from_ahanpakhsh(urls["AhanPakhsh"])

    size_index = size_combobox.current()

    if prices and size_index < len(prices):
        price = prices[size_index]
        prices_label.config(text=f"Price for size {selected_size}: {price}")
    else:
        prices_label.config(text="No price found for this size.")

    # Store the price for use in the second tab
    price_entry.delete(0, tk.END)
    price_entry.insert(0, prices[size_index])

# Function to calculate total price
def calculate_total_price():
    play_click_sound()
    try:
        # Remove any non-numeric characters (except dot for decimal numbers)
        price_text = ''.join(c for c in price_entry.get() if c.isdigit() or c == '.')
        weight_text = ''.join(c for c in weight_entry.get() if c.isdigit() or c == '.')

        if not price_text or not weight_text:
            total_label.config(text="Please enter valid numbers!")
            return

        price = float(price_text)
        weight = float(weight_text)
        total = price * weight
        total_label.config(text=f"Total Price: {total:,.0f} Toman")

    except ValueError:
        total_label.config(text="Invalid input! Enter numbers only.")


def reset_fields():
    website_combobox.set("Select Website")
    size_combobox.set("Select Size")
    weight_entry.delete(0, tk.END)
    price_entry.delete(0, tk.END)
    total_label.config(text="")

In [ ]:
# Create main window
root = ThemedTk()
root.set_theme("breeze")
root.iconbitmap("Logo.ico")
root.title("Steel Total Price")
root.geometry("500x500")

# Create Notebook (tabs)
notebook = ttk.Notebook(root)
notebook.pack(expand=True, fill="both")

# First tab (Price Extraction)
tab1 = ttk.Frame(notebook)
notebook.add(tab1, text="Price Finder")

instruction_label = tk.Label(tab1, text="Select a website and size of rebar:")
instruction_label.pack(pady=10)

website_combobox = ttk.Combobox(tab1, values=["AhanOnline", "Pivan", "AhanPakhsh"])
website_combobox.set("Select Website")
website_combobox.pack(pady=10)

size_combobox = ttk.Combobox(tab1, values=["12", "14", "16", "18", "20", "22", "25", "28", "32"])
size_combobox.set("Select Size")
size_combobox.pack(pady=10)

show_button = tk.Button(tab1, text="Show Price", command=show_prices)
show_button.pack(pady=10)
show_button.bind("<Enter>", lambda e: show_button.config(bg="#00bfff"))
show_button.bind("<Leave>", lambda e: show_button.config(bg="#F0F0F0"))

prices_label = tk.Label(tab1, text="", justify=tk.LEFT)
prices_label.pack(pady=20)


reset_button = tk.Button(root, text="Reset", command=reset_fields)
reset_button.pack(pady=10)

# Second tab (Weight Calculation)
tab2 = ttk.Frame(notebook)
notebook.add(tab2, text="Total Price Calculator")

tk.Label(tab2, text="Enter Weight (kg):").pack(pady=5)
weight_entry = tk.Entry(tab2)
weight_entry.pack(pady=5)

tk.Label(tab2, text="Price per kg:").pack(pady=5)
price_entry = tk.Entry(tab2)
price_entry.pack(pady=5)

calculate_button = tk.Button(tab2, text="Calculate Total Price", command=calculate_total_price)
calculate_button.pack(pady=10)
calculate_button.bind("<Enter>", lambda e: calculate_button.config(bg="#00bfff"))
calculate_button.bind("<Leave>", lambda e: calculate_button.config(bg="#F0F0F0"))

total_label = tk.Label(tab2, text="", font=("Arial", 12, "bold"))
total_label.pack(pady=10)

root.mainloop()


At this stage, we use the `tkinter` library to define a UI window for the software.  

Then, using `ttk.Notebook`, we create tabs for the application.  

Each tab contains buttons that, when clicked, call the corresponding function to either display the price or calculate the total price.